<a href="https://colab.research.google.com/github/mardenlelis/ConfusionMatrixFunction/blob/main/AorticValveSegmentator_vGemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AorticValveSegmentator - Notebook Melhorado

Este notebook foi aprimorado com base nas sugestões de melhoria para o pipeline de segmentação da válvula aórtica usando nnU-Netv2, incluindo correções de caminho, mensagens mais claras e preparação para lidar com erros de memória durante o treinamento.

🚀 CÉLULA 1 — Montar Google Drive

In [1]:
# 📦 Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


🔧 Célula 2 — Instalar dependências

In [2]:
# 📌 Instalar dependências
# ✅ Instalar bibliotecas essenciais e nnU-Net v2
!pip install --upgrade pip
!pip install numpy pandas nibabel tqdm SimpleITK openpyxl --quiet
!pip install medpy --quiet
!pip install nnunetv2 --quiet
# A flag --index-url é crucial para garantir a instalação de uma versão de torch compatível com a GPU (cu118)
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 75.6 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 MB 121.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  DEPRECATION: Building 'medpy' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'medpy'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.p

⚙️ CÉLULA 3 — Verificar GPU e dispositivo

In [3]:
# ⚙️ Verificar disponibilidade de GPU
import torch
print("CUDA disponível:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Dispositivo:", torch.cuda.get_device_name(0))
else:
    print("AVISO: Nenhuma GPU CUDA detectada. O treinamento será muito lento.")

CUDA disponível: True
Dispositivo: Tesla T4


🧹 Célula 4 — Limpar diretórios de origem de imagens e rótulos (opcional)

In [4]:
# AVISO: Esta célula apagará **TODOS** os arquivos nas pastas de imagens e rótulos originais
# dentro da estrutura do nnU-Net. Use com cautela se você já tiver dados lá.
# É útil para recomeçar do zero, mas pode ser comentada se os dados já estiverem preparados.

import os
import shutil

dir_originais_imagens = '/content/drive/MyDrive/nnunet_data/nnUNet_raw/Dataset001_AorticValve/originais/imagens'
dir_originais_rotulos = '/content/drive/MyDrive/nnunet_data/nnUNet_raw/Dataset001_AorticValve/originais/rotulos'

print("Limpando diretórios de origem para Dataset001_AorticValve/originais/imagens e rotulos...")

for d in [dir_originais_imagens, dir_originais_rotulos]:
    if os.path.exists(d):
        # Remove o conteúdo, mas mantém o diretório para recriação se necessário
        for f in os.listdir(d):
            file_path = os.path.join(d, f)
            try:
                if os.path.isfile(file_path) or os.path.islink(file_path):
                    os.unlink(file_path)
                elif os.path.isdir(file_path):
                    shutil.rmtree(file_path)
            except Exception as e:
                print(f'Falha ao deletar {file_path}. Razão: {e}')
    else:
        print(f'Diretório não encontrado: {d}. Criando...')
        os.makedirs(d, exist_ok=True)

print("Limpeza concluída.")


Limpando diretórios de origem para Dataset001_AorticValve/originais/imagens e rotulos...
Limpeza concluída.


📁 Célula 5 - Copia os arquivos do diretório origem e transfere para a estrutura de arquivos nnU-Net

In [5]:
# Essa célula copia os arquivos do diretório origem e transfere para a estrutura de arquivos
# utilizadas pelas células de preparação do dataset, com renomeação específica.

import os
import shutil

# --- Caminhos ---
# ATENÇÃO: Ajuste 'dir_origem' para o local dos seus arquivos NRRD de origem.
dir_origem = '/content/drive/MyDrive/TC_DICOM/nrrd'

# Diretórios de destino para as imagens e os rótulos (labels) na estrutura nnU-Net_raw
dir_destino_imagens = '/content/drive/MyDrive/nnunet_data/nnUNet_raw/Dataset001_AorticValve/originais/imagens'
dir_destino_labels = '/content/drive/MyDrive/nnunet_data/nnUNet_raw/Dataset001_AorticValve/originais/rotulos'

# Garante que os diretórios de destino existam
os.makedirs(dir_destino_imagens, exist_ok=True)
os.makedirs(dir_destino_labels, exist_ok=True)

print("Iniciando a busca, cópia e RENOMEAÇÃO com critérios específicos...")

if not os.path.isdir(dir_origem):
    print(f"\nERRO: O diretório de origem '{dir_origem}' não foi encontrado. Por favor, verifique o caminho e a montagem do Drive.")
else:
    found_any_file = False
    for raiz, subpastas, arquivos in os.walk(dir_origem):
        if raiz == dir_origem: # Pula o diretório raiz em si
            continue

        id_paciente_str = os.path.basename(raiz)
        if not id_paciente_str.isdigit():
            print(f"  AVISO: Ignorando pasta '{raiz}' pois o nome não é um ID de paciente numérico válido.")
            continue

        id_paciente_formatado = id_paciente_str.zfill(3)
        print(f"\nAnalisando pasta do paciente: {id_paciente_formatado}")

        label_copiado = False
        imagem_copiada = False

        for nome_arquivo in arquivos:
            caminho_completo_origem = os.path.join(raiz, nome_arquivo)

            # 1. Procura pelo arquivo de rótulo que termina com '.seg.nrrd'
            if nome_arquivo.endswith('.seg.nrrd') and not label_copiado:
                novo_nome_label = f"{id_paciente_formatado}.nrrd"
                caminho_completo_destino = os.path.join(dir_destino_labels, novo_nome_label)
                print(f"  -> RÓTULO encontrado e renomeado para: {novo_nome_label}")
                shutil.copy2(caminho_completo_origem, caminho_completo_destino)
                label_copiado = True
                found_any_file = True

            # 2. Procura pelo arquivo de imagem com nome exato '120 KV.nrrd'
            if nome_arquivo.endswith('120 KV.nrrd') and not imagem_copiada:
                novo_nome_imagem = f"{id_paciente_formatado}.nrrd"
                caminho_completo_destino = os.path.join(dir_destino_imagens, novo_nome_imagem)
                print(f"  -> IMAGEM encontrada e renomeada para: {novo_nome_imagem}")
                shutil.copy2(caminho_completo_origem, caminho_completo_destino)
                imagem_copiada = True
                found_any_file = True

        if not label_copiado:
            print(f"  -> AVISO: Nenhum arquivo de RÓTULO terminando com '.seg.nrrd' foi encontrado para o paciente {id_paciente_formatado}.")
        if not imagem_copiada:
            print(f"  -> AVISO: Nenhum arquivo de IMAGEM terminando com '120 KV.nrrd' foi encontrado para o paciente {id_paciente_formatado}.")

    if not found_any_file:
        print("\nAVISO: Nenhuma imagem ou rótulo válido foi encontrado no diretório de origem ou subdiretórios.\nCertifique-se de que o caminho está correto e os arquivos seguem o padrão esperado.")

    print("\n\nProcesso de cópia e renomeação concluído!")

Iniciando a busca, cópia e RENOMEAÇÃO com critérios específicos...

Analisando pasta do paciente: 083
  -> IMAGEM encontrada e renomeada para: 083.nrrd
  -> RÓTULO encontrado e renomeado para: 083.nrrd

Analisando pasta do paciente: 084
  -> RÓTULO encontrado e renomeado para: 084.nrrd
  -> IMAGEM encontrada e renomeada para: 084.nrrd

Analisando pasta do paciente: 091
  -> IMAGEM encontrada e renomeada para: 091.nrrd
  -> RÓTULO encontrado e renomeado para: 091.nrrd

Analisando pasta do paciente: 089
  -> RÓTULO encontrado e renomeado para: 089.nrrd
  -> IMAGEM encontrada e renomeada para: 089.nrrd

Analisando pasta do paciente: 090
  -> RÓTULO encontrado e renomeado para: 090.nrrd
  -> IMAGEM encontrada e renomeada para: 090.nrrd

Analisando pasta do paciente: 092
  -> IMAGEM encontrada e renomeada para: 092.nrrd
  -> RÓTULO encontrado e renomeado para: 092.nrrd

Analisando pasta do paciente: 087
  -> RÓTULO encontrado e renomeado para: 087.nrrd
  -> IMAGEM encontrada e renomeada par

📁 Célula 6 — Gerar dataset.json automaticamente e estruturar o dataset para nnU-Net

In [6]:
# ✅ Preparação COMPLETA do dataset nnUNetv2 com limpeza e estrutura correta
!pip install -q SimpleITK

import os
import shutil
import json
import SimpleITK as sitk
from tqdm import tqdm

# ============================================================================
# 1. CONFIGURAÇÃO DE DIRETÓRIOS E LIMPEZA DOS DIRETÓRIOS DE DESTINO
# ============================================================================

DATASET_NAME = "AorticValve"
DATASET_ID = "001"
BASE_DIR = f"/content/drive/MyDrive/nnunet_data/nnUNet_raw/Dataset{DATASET_ID}_{DATASET_NAME}"

SOURCE_IMAGES_DIR = os.path.join(BASE_DIR, "originais", "imagens")
SOURCE_LABELS_DIR = os.path.join(BASE_DIR, "originais", "rotulos")

IMAGES_TR_DIR = os.path.join(BASE_DIR, "imagesTr")
LABELS_TR_DIR = os.path.join(BASE_DIR, "labelsTr")
IMAGES_TS_DIR = os.path.join(BASE_DIR, "imagesTs")
LABELS_TS_DIR = os.path.join(BASE_DIR, "labelsTs")  # Pasta para rótulos do conjunto de teste (para avaliação)

# 🔄 LIMPEZA DOS DIRETÓRIOS nnU-Net
print("🧹 Limpando e recriando diretórios de destino nnU-Net...")
for path in [IMAGES_TR_DIR, LABELS_TR_DIR, IMAGES_TS_DIR, LABELS_TS_DIR]:
    if os.path.exists(path): # Remove o diretório se já existir
        shutil.rmtree(path)
    os.makedirs(path, exist_ok=True) # Cria o diretório limpo

# ============================================================================
# 2. FUNÇÃO DE CONVERSÃO CORRETA PARA .nii.gz COM SimpleITK
# ============================================================================

def convert_and_save_gzipped(source_path, dest_path):
    if not os.path.exists(source_path):
        print(f"  ⚠️ Arquivo não encontrado: {os.path.basename(source_path)}. Pulando a conversão.")
        return False
    try:
        img = sitk.ReadImage(source_path)
        sitk.WriteImage(img, dest_path, useCompression=True)
        return True
    except Exception as e:
        print(f"  ❌ Erro ao converter {os.path.basename(source_path)} para NIfTI: {e}")
        return False

# ============================================================================
# 3. CONVERSÃO DAS IMAGENS E DIVISÃO TREINO/TESTE
# ============================================================================

print("🔎 Lendo arquivos de origem...")
image_files_unsorted = [f for f in os.listdir(SOURCE_IMAGES_DIR) if not f.startswith('.')]
label_files_unsorted = [f for f in os.listdir(SOURCE_LABELS_DIR) if not f.startswith('.')]

# Cria uma lista de tuplas (id_paciente, nome_arquivo_imagem, nome_arquivo_label)
paired_files = []
for img_fname in image_files_unsorted:
    base_name = img_fname.split('.')[0] # Assume que o nome base é o ID do paciente
    label_fname = f"{base_name}.nrrd"
    if label_fname in label_files_unsorted:
        paired_files.append((base_name, img_fname, label_fname))
    else:
        print(f"  AVISO: Rótulo '{label_fname}' não encontrado para a imagem '{img_fname}'. Ignorando este par.")

paired_files.sort(key=lambda x: x[0]) # Ordena para reprodutibilidade

num_total_valid_files = len(paired_files)
if num_total_valid_files == 0:
    print("🚨 ERRO: Nenhum par de imagem/rótulo válido encontrado nos diretórios de origem. Certifique-se de que a Célula 5 foi executada corretamente.")
    exit() # Sai do script se não houver dados

num_train_desired = min(80, num_total_valid_files)
num_test_desired = num_total_valid_files - num_train_desired # O restante vai para teste

num_train_files = 0
num_test_files = 0
train_metadata = []
test_metadata = []

print(f"🚀 Processando {num_train_desired} casos para treino...")
for i in tqdm(range(num_train_desired)):
    base_name, original_image_fname, original_label_fname = paired_files[i]
    original_image_path = os.path.join(SOURCE_IMAGES_DIR, original_image_fname)
    original_label_path = os.path.join(SOURCE_LABELS_DIR, original_label_fname)

    case_id_nnunet = f"{DATASET_NAME}_{base_name}" # ID no formato nnU-Net

    image_out_path = os.path.join(IMAGES_TR_DIR, f"{case_id_nnunet}_0000.nii.gz")
    label_out_path = os.path.join(LABELS_TR_DIR, f"{case_id_nnunet}.nii.gz")

    img_ok = convert_and_save_gzipped(original_image_path, image_out_path)
    lbl_ok = convert_and_save_gzipped(original_label_path, label_out_path)

    if img_ok and lbl_ok:
        num_train_files += 1
        train_metadata.append({
            "image": f"./imagesTr/{case_id_nnunet}_0000.nii.gz",
            "label": f"./labelsTr/{case_id_nnunet}.nii.gz"
        })
    else:
        print(f"  AVISO: Caso {base_name} não foi incluído no treino devido a erro na conversão.")

print(f"\n🧪 Processando {num_test_desired} casos para teste...")
for i in tqdm(range(num_train_desired, num_total_valid_files)):
    base_name, original_image_fname, original_label_fname = paired_files[i]
    original_image_path = os.path.join(SOURCE_IMAGES_DIR, original_image_fname)
    original_label_path = os.path.join(SOURCE_LABELS_DIR, original_label_fname)

    case_id_nnunet = f"{DATASET_NAME}_{base_name}"

    image_out_path = os.path.join(IMAGES_TS_DIR, f"{case_id_nnunet}_0000.nii.gz")
    label_out_path = os.path.join(LABELS_TS_DIR, f"{case_id_nnunet}.nii.gz") # Copiar label para labelsTs para avaliação posterior

    img_ok = convert_and_save_gzipped(original_image_path, image_out_path)
    lbl_ok = convert_and_save_gzipped(original_label_path, label_out_path)

    if img_ok:
        num_test_files += 1
        test_metadata.append(f"./imagesTs/{case_id_nnunet}_0000.nii.gz")
    else:
        print(f"  AVISO: Caso {base_name} não foi incluído no teste devido a erro na conversão.")


# ============================================================================
# 4. GERAR dataset.json COMPLETO (VERSÃO FINAL nnUNetv2)
# ============================================================================

print("\n📝 Gerando dataset.json...")

dataset_info = {
    "channel_names": {
        "0": "CT" # Nome do canal de entrada (CT, MR, etc.)
    },
    "labels": {
        "background": "0",
        "aortic_valve": "1" # Mapeamento do rótulo 1 para 'aortic_valve'
    },
    "modality": {
        "0": "CT"
    },
    "file_ending": ".nii.gz",
    "numTraining": num_train_files,
    "training": train_metadata,
    "test": test_metadata,
    "name": DATASET_NAME, # Nome do dataset
    "description": "Segmentação da Válvula Aórtica em Tomografia Computadorizada",
    "reference": "N/A",
    "licence": "N/A",
    "release": "0.0"
}

with open(os.path.join(BASE_DIR, "dataset.json"), "w", encoding="utf-8") as f:
    json.dump(dataset_info, f, indent=4)

print("\n✅ Dataset estruturado com sucesso!")
print(f"✔️ Treino: {num_train_files} casos")
print(f"✔️ Teste : {num_test_files} casos (com labelsTs para avaliação)")
print(f"📁 Arquivo JSON: {os.path.join(BASE_DIR, 'dataset.json')}")


🧹 Limpando e recriando diretórios de destino nnU-Net...
🔎 Lendo arquivos de origem...
🚀 Processando 80 casos para treino...


100%|██████████| 80/80 [01:10<00:00,  1.14it/s]



🧪 Processando 20 casos para teste...


100%|██████████| 20/20 [00:17<00:00,  1.14it/s]



📝 Gerando dataset.json...

✅ Dataset estruturado com sucesso!
✔️ Treino: 80 casos
✔️ Teste : 20 casos (com labelsTs para avaliação)
📁 Arquivo JSON: /content/drive/MyDrive/nnunet_data/nnUNet_raw/Dataset001_AorticValve/dataset.json


In [ ]:
# ✅ ADICIONAR NOVA CÉLULA 6B - Augmentation Avançada:

import os
import shutil
import numpy as np
import SimpleITK as sitk
from scipy.ndimage import rotate, gaussian_filter

def advanced_augmentation(image_path, label_path, output_dir_img, output_dir_lbl, base_name):
    """Aplica augmentations avançadas específicas para válvula aórtica"""

    # Carregar imagem e label
    img = sitk.ReadImage(image_path)
    lbl = sitk.ReadImage(label_path)

    img_array = sitk.GetArrayFromImage(img)
    lbl_array = sitk.GetArrayFromImage(lbl)

    augmented_pairs = []

    # 1. Rotações pequenas (válvula aórtica é sensível a rotação)
    for angle in [-5, 5, -10, 10]:
        img_rot = rotate(img_array, angle, axes=(1, 2), reshape=False, order=1)
        lbl_rot = rotate(lbl_array, angle, axes=(1, 2), reshape=False, order=0)
        augmented_pairs.append((img_rot, lbl_rot, f"{base_name}_rot{angle}"))

    # 2. Flip horizontal/vertical
    img_flip_h = np.flip(img_array, axis=2)
    lbl_flip_h = np.flip(lbl_array, axis=2)
    augmented_pairs.append((img_flip_h, lbl_flip_h, f"{base_name}_flipH"))

    # 3. Gaussian noise (simula ruído de CT)
    noise_std = np.std(img_array) * 0.05
    img_noise = img_array + np.random.normal(0, noise_std, img_array.shape)
    augmented_pairs.append((img_noise, lbl_array, f"{base_name}_noise"))

    # 4. Contraste ajustado (importante para CT)
    img_contrast = np.clip(img_array * 1.2 - 50, img_array.min(), img_array.max())
    augmented_pairs.append((img_contrast, lbl_array, f"{base_name}_contrast"))

    # Salvar todas as versões aumentadas
    for img_aug, lbl_aug, suffix in augmented_pairs:
        # Criar novas imagens SimpleITK
        img_aug_sitk = sitk.GetImageFromArray(img_aug)
        lbl_aug_sitk = sitk.GetImageFromArray(lbl_aug.astype(np.uint8))

        # Copiar metadados originais
        img_aug_sitk.CopyInformation(img)
        lbl_aug_sitk.CopyInformation(lbl)

        # Salvar
        sitk.WriteImage(img_aug_sitk, os.path.join(output_dir_img, f"{suffix}_0000.nii.gz"), useCompression=True)
        sitk.WriteImage(lbl_aug_sitk, os.path.join(output_dir_lbl, f"{suffix}.nii.gz"), useCompression=True)

# Aplicar augmentation apenas aos dados de treino
print("🔄 Aplicando data augmentation avançada...")

SOURCE_IMAGES_DIR = os.path.join(BASE_DIR, "originais", "imagens")
SOURCE_LABELS_DIR = os.path.join(BASE_DIR, "originais", "rotulos")

# Pegar apenas os primeiros 80% para treino (mesma lógica da célula original)
image_files = sorted([f for f in os.listdir(SOURCE_IMAGES_DIR) if not f.startswith('.')])
num_train = min(80, len(image_files))

for i in range(num_train):
    fname = image_files[i]
    base_name = fname.split('.')[0]

    img_path = os.path.join(SOURCE_IMAGES_DIR, fname)
    lbl_path = os.path.join(SOURCE_LABELS_DIR, fname)

    if os.path.exists(img_path) and os.path.exists(lbl_path):
        try:
            advanced_augmentation(img_path, lbl_path, IMAGES_TR_DIR, LABELS_TR_DIR, f"AorticValve_{base_name}")
            print(f"  ✓ Augmentado: {base_name}")
        except Exception as e:
            print(f"  ❌ Erro no {base_name}: {e}")

print("✅ Data augmentation concluída!")

🔁 CÉLULA 7 — Preparação e Treinamento do nnU-Netv2

In [7]:
# ✅ Definindo os paths globais do nnUNetv2
import os
import torch # Necessário para configurar variáveis de ambiente PyTorch

os.environ["nnUNet_raw"] = "/content/drive/MyDrive/nnunet_data/nnUNet_raw"
os.environ["nnUNet_preprocessed"] = "/content/drive/MyDrive/nnunet_data/nnUNet_preprocessed"
os.environ["nnUNet_results"] = "/content/drive/MyDrive/nnunet_data/nnUNet_results"

# =========================================================================
# SOLUÇÃO PARA OUT OF MEMORY (OOM):
# 1. Tentar alocação de memória expansível (Pytorch)
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
# 2. Se o erro de OOM persistir, você DEVE reduzir o batch_size e/ou patch_size.
#    Para fazer isso, execute a célula abaixo para gerar os planos (se ainda não fez).
#    Em seguida, VÁ MANUALMENTE ao arquivo:
#    /content/drive/MyDrive/nnunet_data/nnUNet_preprocessed/Dataset001_AorticValve/nnUNetResEncUNetLPlans.json
#    Edite a seção '3d_fullres' e altere:
#    "batch_size": 2  ->  "batch_size": 1  (ou menos, se necessário)
#    "patch_size": [32, 384, 384] -> [24, 320, 320] (ou tamanhos menores, se necessário)
#    APÓS A EDIÇÃO, NÃO EXECUTE NOVAMENTE a etapa de 'plan_and_preprocess' pois ela sobrescreverá suas alterações!
# =========================================================================

print("--- Etapa de Planejamento e Pré-processamento ---")
print("Isso gerará os planos de treinamento e pré-processará os dados.")
print("Se você já editou o arquivo de planos manualmente (para OOM), NÃO execute esta linha novamente.")

# Gerar planos e pré-processar o dataset
# -d 001: ID do dataset
# -pl nnUNetPlannerResEncL: Usa o plano de arquitetura Residual Encoder UNet L
!nnUNetv2_plan_and_preprocess -d 001 -pl nnUNetPlannerResEncL

print("\n--- Início do Treinamento nnUNetv2 ---")
print("Certifique-se de que o arquivo nnUNetResEncUNetLPlans.json foi ajustado se encontrou OutOfMemoryError.")

# Iniciar o treinamento
# 001: ID do dataset
# 3d_fullres: Configuração de treinamento (3D full resolution)
# 0: Fold de validação cruzada (usando a fold 0). Para treinamento completo, idealmente, você treinaria 5 folds.
# -p nnUNetResEncUNetLPlans: Especifica o plano de treinamento a ser usado
!nnUNetv2_train 001 3d_fullres 0 -p nnUNetResEncUNetLPlans

print("\nTreinamento concluído (ou falhou devido a OOM - verifique a saída acima)!")

--- Etapa de Planejamento e Pré-processamento ---
Isso gerará os planos de treinamento e pré-processará os dados.
Se você já editou o arquivo de planos manualmente (para OOM), NÃO execute esta linha novamente.
Fingerprint extraction...
Dataset001_AorticValve
Experiment planning...
Dropping 3d_lowres config because the image size difference to 3d_fullres is too small. 3d_fullres: [ 50. 512. 512.], 3d_lowres: [50, 512, 512]
2D U-Net configuration:
{'data_identifier': 'nnUNetPlans_2d', 'preprocessor_name': 'DefaultPreprocessor', 'batch_size': 35, 'patch_size': (np.int64(512), np.int64(512)), 'median_image_size_in_voxels': array([512., 512.]), 'spacing': array([0.35058594, 0.35058594]), 'normalization_schemes': ['CTNormalization'], 'use_mask_for_norm': [False], 'resampling_fn_data': 'resample_data_or_seg_to_shape', 'resampling_fn_seg': 'resample_data_or_seg_to_shape', 'resampling_fn_data_kwargs': {'is_seg': False, 'order': 3, 'order_z': 0, 'force_separate_z': None}, 'resampling_fn_seg_kwar

In [ ]:
# Iniciar o treinamento
# 001: ID do dataset
# 3d_fullres: Configuração de treinamento (3D full resolution)
# 0: Fold de validação cruzada (usando a fold 0). Para treinamento completo, idealmente, você treinaria 5 folds.
# -p nnUNetResEncUNetLPlans: Especifica o plano de treinamento a ser usado
!nnUNetv2_train 001 3d_fullres 0 -p nnUNetResEncUNetLPlans

Using device: cuda:0

#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################

2025-07-16 16:43:26.086724: Using torch.compile...
2025-07-16 16:43:27.030293: do_dummy_2d_data_aug: True
2025-07-16 16:43:27.036246: Using splits from existing split file: /content/drive/MyDrive/nnunet_data/nnUNet_preprocessed/Dataset001_AorticValve/splits_final.json
2025-07-16 16:43:27.040446: The split file contains 5 splits.
2025-07-16 16:43:27.042759: Desired fold for training: 0
2025-07-16 16:43:27.045249: This split has 64 training and 16 validation cases.
using pin_memory on device 0
using pin_memory on device 0

This is the configuration used by this training:


In [ ]:
# ✅ ADICIONAR NOVA CÉLULA 7B - Modificar Planos para Performance:

import json
import os

# Caminho do arquivo de planos
plans_file = "/content/drive/MyDrive/nnunet_data/nnUNet_preprocessed/Dataset001_AorticValve/nnUNetResEncUNetLPlans.json"

if os.path.exists(plans_file):
    print("📝 Modificando planos para melhor performance...")

    with open(plans_file, 'r') as f:
        plans = json.load(f)

    # MELHORIAS CRÍTICAS:
    if '3d_fullres' in plans['configurations']:
        config = plans['configurations']['3d_fullres']

        # 1. Aumentar épocas
        config['num_epochs'] = 1000  # Era ~250, agora 1000

        # 2. Reduzir batch size se OOM
        if config.get('batch_size', 2) > 1:
            config['batch_size'] = 1
            print("  ✓ Batch size reduzido para 1")

        # 3. Otimizar patch size para sua GPU
        original_patch = config.get('patch_size', [32, 384, 384])
        config['patch_size'] = [24, 320, 320]  # Menor para evitar OOM
        print(f"  ✓ Patch size: {original_patch} → {config['patch_size']}")

        # 4. Adicionar early stopping mais agressivo
        config['patience'] = 50  # Parar após 50 épocas sem melhoria

    # Salvar modificações
    with open(plans_file, 'w') as f:
        json.dump(plans, f, indent=2)

    print("✅ Planos modificados com sucesso!")
else:
    print("⚠️ Arquivo de planos não encontrado. Execute primeiro o plan_and_preprocess.")

🔮 CÉLULA 8 — Predição nos dados de teste

In [ ]:
# ✅ Predição nos dados de teste
import os

DATASET_ID = "001"
DATASET_NAME = "AorticValve"
TRAINER_PLANS = "nnUNetTrainer_plans_nnUNetResEncUNetLPlans"
CONFIGURATION = "3d_fullres"
FOLD = "0"

INPUT_TEST_DIR = f"/content/drive/MyDrive/nnunet_data/nnUNet_raw/Dataset{DATASET_ID}_{DATASET_NAME}/imagesTs"
OUTPUT_PREDS_DIR_TEST = f"/content/drive/MyDrive/nnunet_data/nnUNet_results/Dataset{DATASET_ID}_{DATASET_NAME}/{TRAINER_PLANS}_{CONFIGURATION}/fold_{FOLD}/predictionsTs_ResEncL"

print(f"Iniciando predição nos dados de teste de {INPUT_TEST_DIR}...")
print(f"Resultados serão salvos em: {OUTPUT_PREDS_DIR_TEST}")

# O nnUNetv2_predict criará os diretórios de saída automaticamente.
!nnUNetv2_predict \
  -d {DATASET_ID} \
  -i {INPUT_TEST_DIR} \
  -o {OUTPUT_PREDS_DIR_TEST} \
  -c {CONFIGURATION} \
  -f {FOLD} \
  -p nnUNetResEncUNetLPlans \
  -chk checkpoint_best.pth # Usa o melhor checkpoint de validação

🔮 CÉLULA 9 — Predição nos dados de treino (validação cruzada)

In [ ]:
# ✅ Predição nos dados de treino (validação cruzada)
import os

DATASET_ID = "001"
DATASET_NAME = "AorticValve"
TRAINER_PLANS = "nnUNetTrainer_plans_nnUNetResEncUNetLPlans"
CONFIGURATION = "3d_fullres"
FOLD = "0"

INPUT_TRAIN_DIR = f"/content/drive/MyDrive/nnunet_data/nnUNet_raw/Dataset{DATASET_ID}_{DATASET_NAME}/imagesTr"
OUTPUT_PREDS_DIR_TRAIN = f"/content/drive/MyDrive/nnunet_data/nnUNet_results/Dataset{DATASET_ID}_{DATASET_NAME}/{TRAINER_PLANS}_{CONFIGURATION}/fold_{FOLD}/predictionsTr_ResEncL"

print(f"Iniciando predição nos dados de treino (validação) de {INPUT_TRAIN_DIR}...")
print(f"Resultados serão salvos em: {OUTPUT_PREDS_DIR_TRAIN}")

!nnUNetv2_predict \
  -d {DATASET_ID} \
  -i {INPUT_TRAIN_DIR} \
  -o {OUTPUT_PREDS_DIR_TRAIN} \
  -c {CONFIGURATION} \
  -f {FOLD} \
  -p nnUNetResEncUNetLPlans \
  -chk checkpoint_best.pth # Usa o melhor checkpoint de validação

🧠 CÉLULA 10 — Identificar cálcio apenas dentro da válvula predita (para os dados de teste)

In [ ]:
# ✅ Extrair apenas o cálcio dentro da válvula nas predições dos casos de teste
import nibabel as nib
import numpy as np
import os

DATASET_ID = "001"
DATASET_NAME = "AorticValve"
TRAINER_PLANS = "nnUNetTrainer_plans_nnUNetResEncUNetLPlans"
CONFIGURATION = "3d_fullres"
FOLD = "0"

BASE_NNUNET_RAW_DIR = f"/content/drive/MyDrive/nnunet_data/nnUNet_raw/Dataset{DATASET_ID}_{DATASET_NAME}"
BASE_NNUNET_RESULTS_DIR = f"/content/drive/MyDrive/nnunet_data/nnUNet_results/Dataset{DATASET_ID}_{DATASET_NAME}/{TRAINER_PLANS}_{CONFIGURATION}/fold_{FOLD}"

images_dir = os.path.join(BASE_NNUNET_RAW_DIR, 'imagesTs')
preds_valve_dir = os.path.join(BASE_NNUNET_RESULTS_DIR, 'predictionsTs_ResEncL') # Predições da válvula aórtica
output_calcium_dir = os.path.join(BASE_NNUNET_RESULTS_DIR, 'predictionsTs_calcio_na_valvula') # Onde o cálcio filtrado será salvo

os.makedirs(output_calcium_dir, exist_ok=True)

HU_LIMIAR_CALCIFICACAO = 130  # Limiar de unidades Hounsfield para identificar cálcio

print(f"Processando imagens de: {images_dir}")
print(f"Usando predições da válvula de: {preds_valve_dir}")
print(f"Salvando resultados do cálcio filtrado em: {output_calcium_dir}
")

processed_count = 0
for fname in sorted(os.listdir(images_dir)):
    if not fname.endswith('_0000.nii.gz'): # Busca arquivos de imagem (terminam com _0000.nii.gz)
        continue

    # Extrai o ID do paciente (ex: 'AorticValve_083' de 'AorticValve_083_0000.nii.gz')
    id_paciente_base = fname.replace('_0000.nii.gz', '')
    print(f"🔍 Processando paciente: {id_paciente_base}")

    try:
        img_path = os.path.join(images_dir, fname)
        pred_valve_path = os.path.join(preds_valve_dir, f"{id_paciente_base}.nii.gz")

        if not os.path.exists(img_path):
            print(f"  AVISO: Imagem '{img_path}' não encontrada. Pulando este paciente.")
            continue
        if not os.path.exists(pred_valve_path):
            print(f"  AVISO: Predição da válvula '{pred_valve_path}' não encontrada. Pulando este paciente.")
            continue

        img = nib.load(img_path)
        img_data = img.get_fdata()
        pred_valve = nib.load(pred_valve_path).get_fdata()

        valve_mask = pred_valve > 0 # Cria máscara booleana da válvula predita (valores > 0 são válvula)
        calc_mask = img_data > HU_LIMIAR_CALCIFICACAO # Cria máscara booleana do cálcio na imagem original

        # O cálcio dentro da válvula é a intersecção das duas máscaras
        calc_dentro_valvula = np.logical_and(valve_mask, calc_mask)

        # Salva o resultado como um novo arquivo NIfTI
        nifti_out = nib.Nifti1Image(calc_dentro_valvula.astype(np.uint8), img.affine, img.header)
        nib.save(nifti_out, os.path.join(output_calcium_dir, f"{id_paciente_base}.nii.gz"))
        processed_count += 1
    except Exception as e:
        print(f"  ❌ Erro ao processar paciente {id_paciente_base}: {e}")

print(f"\n✅ Cálcio na válvula (teste) salvo em: {output_calcium_dir} ({processed_count} arquivos processados)")

🧠 CÉLULA 11 — Avaliação Dice + HD95 dos cálcios detectados (em teste)

In [ ]:
# ✅ Avaliação Dice e HD95 do cálcio detectado na válvula predita (em comparação com o rótulo da válvula)
# NOTA IMPORTANTE: Esta métrica compara a máscara de cálcio *predita dentro da válvula predita*
# com o *rótulo original da válvula*. Se você possui rótulos específicos para o cálcio (ground truth)
# eles deveriam ser usados para uma avaliação direta da segmentação de cálcio.

import pandas as pd
from medpy.metric import binary
import nibabel as nib
import numpy as np
import os

DATASET_ID = "001"
DATASET_NAME = "AorticValve"
TRAINER_PLANS = "nnUNetTrainer_plans_nnUNetResEncUNetLPlans"
CONFIGURATION = "3d_fullres"
FOLD = "0"

BASE_NNUNET_RAW_DIR = f"/content/drive/MyDrive/nnunet_data/nnUNet_raw/Dataset{DATASET_ID}_{DATASET_NAME}"
BASE_NNUNET_RESULTS_DIR = f"/content/drive/MyDrive/nnunet_data/nnUNet_results/Dataset{DATASET_ID}_{DATASET_NAME}/{TRAINER_PLANS}_{CONFIGURATION}/fold_{FOLD}"

labels_valve_dir = os.path.join(BASE_NNUNET_RAW_DIR, 'labelsTs') # Rótulos originais da válvula
preds_calcium_in_valve_dir = os.path.join(BASE_NNUNET_RESULTS_DIR, 'predictionsTs_calcio_na_valvula') # Predições do cálcio filtrado

resultados = []
print(f"Avaliando predições de cálcio na válvula em: {preds_calcium_in_valve_dir}")
print(f"Comparando com rótulos da válvula em: {labels_valve_dir}
")

evaluated_count = 0
for fname in sorted(os.listdir(preds_calcium_in_valve_dir)):
    if not fname.endswith('.nii.gz'):
        continue

    patient_id = fname.replace('.nii.gz', '')
    print(f"📊 Avaliando paciente: {patient_id}")

    try:
        pred_calcium_path = os.path.join(preds_calcium_in_valve_dir, fname)
        label_valve_path = os.path.join(labels_valve_dir, fname) # Assume o mesmo nome de arquivo

        if not os.path.exists(pred_calcium_path):
            print(f"  AVISO: Predição de cálcio {pred_calcium_path} não encontrada. Pulando.")
            continue
        if not os.path.exists(label_valve_path):
            print(f"  AVISO: Rótulo da válvula {label_valve_path} não encontrado. Pulando.")
            continue

        pred_mask = nib.load(pred_calcium_path).get_fdata() > 0
        label_mask = nib.load(label_valve_path).get_fdata() > 0

        dice = float('nan')
        hd95 = float('nan')

        # Lida com casos onde uma ou ambas as máscaras podem estar vazias
        if np.sum(pred_mask) == 0 and np.sum(label_mask) == 0:
            dice = 1.0 # Perfeita concordância se ambos vazios
            hd95 = 0.0 # Distância zero se ambos vazios
        elif np.sum(pred_mask) == 0 or np.sum(label_mask) == 0:
            dice = 0.0 # Um vazio e o outro não, sem sobreposição
            # HD95 para um conjunto vazio e outro não é geralmente 'inf' ou um valor grande.
            # MedPy levanta erro para isso, então tratamos.
            hd95 = float('inf')
        else:
            try:
                dice = binary.dc(pred_mask, label_mask)
                hd95 = binary.hd95(pred_mask, label_mask)
            except Exception as metric_e:
                print(f"  AVISO: Erro ao calcular métricas para {patient_id}: {metric_e}. Definindo como NaN.")

        resultados.append({'Paciente': patient_id, 'Dice': round(dice, 4), 'HD95': round(hd95, 2)})
        evaluated_count += 1
    except Exception as e:
        print(f"  ❌ Erro geral ao processar paciente {patient_id} para métricas: {e}")
        resultados.append({'Paciente': patient_id, 'Dice': float('nan'), 'HD95': float('nan')})

df = pd.DataFrame(resultados)
output_csv_path = os.path.join(preds_calcium_in_valve_dir, 'metrics_summary_calcio.csv')
df.to_csv(output_csv_path, index=False)

print(f"\n✅ Sumário das métricas salvo em: {output_csv_path} ({evaluated_count} pacientes avaliados)")
print("\nDataFrame de resultados:")
print(df)


📊 CÉLULA 12 — Visualizações gráficas (Boxplot + Histograma)

In [ ]:
# ✅ Visualizações gráficas das métricas de desempenho
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os

DATASET_ID = "001"
DATASET_NAME = "AorticValve"
TRAINER_PLANS = "nnUNetTrainer_plans_nnUNetResEncUNetLPlans"
CONFIGURATION = "3d_fullres"
FOLD = "0"

BASE_NNUNET_RESULTS_DIR = f"/content/drive/MyDrive/nnunet_data/nnUNet_results/Dataset{DATASET_ID}_{DATASET_NAME}/{TRAINER_PLANS}_{CONFIGURATION}/fold_{FOLD}"
preds_calcium_in_valve_dir = os.path.join(BASE_NNUNET_RESULTS_DIR, 'predictionsTs_calcio_na_valvula')
input_csv_path = os.path.join(preds_calcium_in_valve_dir, 'metrics_summary_calcio.csv')

print(f"Tentando carregar dados de: {input_csv_path}")

if not os.path.exists(input_csv_path):
    print(f"ERRO: Arquivo de métricas '{input_csv_path}' não encontrado.\nCertifique-se de que a Célula 11 foi executada com sucesso e gerou o CSV.")
else:
    df = pd.read_csv(input_csv_path)

    if df.empty:
        print("AVISO: O DataFrame de métricas está vazio. Não há dados para plotar.")
    else:
        sns.set(style="whitegrid")

        # Boxplot Dice
        plt.figure(figsize=(8, 6))
        sns.boxplot(y=df['Dice'])
        plt.title('Distribuição do Coeficiente Dice (Cálcio na Válvula Predita)', fontsize=14)
        plt.ylabel('Coeficiente Dice', fontsize=12)
        plt.grid(True, linestyle='--', alpha=0.7)
        plt.show()

        # Boxplot HD95
        plt.figure(figsize=(8, 6))
        # Remover NaNs para o boxplot, se houver, pois o HD95 pode ser 'inf' para máscaras vazias
        hd95_data = df['HD95'].replace([np.inf, -np.inf], np.nan).dropna() # Remove inf e NaN
        if hd95_data.empty:
            print("AVISO: Não há dados válidos para plotar o Boxplot HD95.")
        else:
            sns.boxplot(y=hd95_data)
            plt.title('Distribuição da Distância de Hausdorff 95 (Cálcio na Válvula Predita)', fontsize=14)
            plt.ylabel('HD95 (mm)', fontsize=12)
            plt.grid(True, linestyle='--', alpha=0.7)
            plt.show()

        # Histograma Dice
        plt.figure(figsize=(10, 7))
        sns.histplot(df['Dice'], bins=15, kde=True, color='skyblue', edgecolor='black')
        plt.title('Histograma do Coeficiente Dice (Cálcio na Válvula Predita)', fontsize=14)
        plt.xlabel('Coeficiente Dice', fontsize=12)
        plt.ylabel('Frequência', fontsize=12)
        plt.grid(True, linestyle='--', alpha=0.7)
        plt.show()

        print("\n✅ Visualizações geradas com sucesso!")
